# Building benchmark models

**Author: Matthew Massett**

Date: 17-03-2025

This notebook provides instructions for constructing BLOSUM and RANDOM benchmark models using a contrived short peptide sequence.

Table of contents
-----------------
1. [Building a tokeniser](#section1)
1. [Building a Blosum sequence model](#section2)
2. [Building a random sequence model](#section3)
3. [Generating variant proteins](#section4)

In [ ]:
#pip install -q prodigyprotein/dist/prodigyprotein-1.0.0.tar.gz

Prodigy Protein has defined Blosum probability embedding layers that can used to obtain the probability of each substitution at all positions. These take as input a protein sequence (tokenised) and return (for every position) the probabilities for substitutions. It operates similar to embedding layers in a sequence model, but with every token being mapped to a a unique probability vector. 

The sampler for variant generation and the BlosumProbabilityEmbedding layer is imported below.

In [ ]:
from prodigyprotein import (
    WeightedDirectedEvolutionSampler, 
    BlosumProbabilityEmbedding
)
import tensorflow as tf

<a id="section1"></a>

## 1. Building a tokeniser

A quick tokeniser is defined below to convert amino acid (AA) residues into integers. AA letters are ordered alphabetically. This tokeniser will not handle sequences of different lengths as both example benchmark models defined here do not have padding tokens.

In [67]:

class blosum_tokeniser():

    def __init__(self):
        
        self.vocab = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
                      'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    
    def __call__(self, input):

        tokenised = []
        sequences = {}

        # Strip whitespace from input strings
        input = [i.replace(' ', '') for i in input]

        # Get max length sequence in input

        for sequence in input:
            
            tokens = tf.constant([self.vocab.index(aa) for aa in sequence])[None,]  
            tokenised.append(tokens)
    
        # Copy tokenised tensor to n_variants
        tokenised = tf.concat(tokenised, axis=0)
        
        sequences['input_ids'] = tokenised
        sequences['attention_mask'] = tf.ones_like(tokenised)
        
        return sequences

    def batch_decode(self,
                     tokenised_sequence,
                     skip_special_tokens=True):
        
        decoded_sequence = []
        
        for tokenised_seq in tokenised_sequence:
            seq = ' '.join([self.vocab[i] for i in tokenised_seq])
            decoded_sequence.append(seq)

        return decoded_sequence

The tokeniser enables the tokenisation of amino acid sequences to integers. Later, it will allow the conversion of generated variants back to AA letters.

In [68]:
tokeniser = blosum_tokeniser()

encoded = tokeniser(['AA', 'AA'])
decoded = tokeniser.batch_decode(encoded['input_ids'])

print(encoded)
print(decoded)

{'input_ids': <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[0, 0],
       [0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1, 1],
       [1, 1]], dtype=int32)>}
['A A', 'A A']



<a id="section2"></a>

## 2. Building a Blosum sequence model



In [69]:
def blosum_model(blosum):
    '''
    Returns Blosum TF model. Detects amino acid in sequence and retrieves probability
    of substitution to the 20 possible amino acids. 
    '''
    input_chains = tf.keras.layers.Input(shape=(None,), name='input_ids', dtype='int32')
    input_ = tf.keras.layers.Input(shape=(None,), name='attention_mask',dtype='int32')
    logits = BlosumProbabilityEmbedding(cluster_pc=blosum)(input_chains)
    logits = tf.cast(logits, tf.float64)

    blosum_model = tf.keras.models.Model(inputs={'input_ids': input_chains, 'attention_mask': input_}, 
                                         outputs={'logits': logits})

    return blosum_model

A blosum sequence model is constructed below. The probabilities returned will be in alphabetical order. **The DirectedEvolution samplers automatically detect whether a model returned probabilities or logits so this can be used immediately in the variant generation workflow.**

Recall, transformers tokenisers return two outputs (default) consisting of the tokenised sequence and an attention mask. To make the model compatible with DirectedEvolution we make the model accept these as well. *Note the attention mask does not change the returned probabilities.*

In [70]:
blosum_p_model = blosum_model(blosum=45)
print(blosum_p_model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, None)]               0         []                            
                                                                                                  
 blosum_probability_embeddi  (None, None, 20)             400       ['input_ids[0][0]']           
 ng_1 (BlosumProbabilityEmb                                                                       
 edding)                                                                                          
                                                                                                  
 attention_mask (InputLayer  [(None, None)]               0         []                            
 )                                                                                          

Lets test it out. Note the model takes a tokenised sequence of length `None`. This is so that it can accept arbitrarily long protein sequences. 

The model returns probabilities of shape `[None, None, 20]` corresponding to the 20 amino acid probabilities for each sequence, position and residue.  

In [71]:
# Sequence of tokens as input. Corresponding to a contrived short peptide sequence alanine, alanine, arginine.
short_peptide = tf.constant([[0, 0, 1]])
attention_mask = tf.constant([[1, 1, 1]])
# Get probabilities from blosum model
peptide_substitution_p = blosum_p_model({
    'input_ids': short_peptide,
    'attention_mask': attention_mask
} )

print(peptide_substitution_p)

{'logits': <tf.Tensor: shape=(1, 3, 20), dtype=float64, numpy=
array([[[0.48088914, 0.00497193, 0.04080733, 0.01465601, 0.02156289,
         0.08338834, 0.00684768, 0.01202223, 0.00868969, 0.02247673,
         0.01091138, 0.0148531 , 0.01872314, 0.00687112, 0.03291445,
         0.07793844, 0.03591704, 0.03769072, 0.01068009, 0.05718854],
        [0.48088914, 0.00497193, 0.04080733, 0.01465601, 0.02156289,
         0.08338834, 0.00684768, 0.01202223, 0.00868969, 0.02247673,
         0.01091138, 0.0148531 , 0.01872314, 0.00687112, 0.03291445,
         0.07793844, 0.03591704, 0.03769072, 0.01068009, 0.05718854],
        [0.07373622, 0.55200917, 0.03539559, 0.00898901, 0.02645044,
         0.03616481, 0.00593956, 0.00737362, 0.00532967, 0.01949593,
         0.00946435, 0.01288332, 0.00812007, 0.00421428, 0.02854943,
         0.04780217, 0.03115383, 0.03269229, 0.00463187, 0.04960437]]])>}


The probabilties returned from the model are the same for the first two positions as expected since they are the same amino acid.

<a id="section3"></a>

## Building a Random sequence model

Lets build a model that returns the same logit for every position and residue.

In [72]:
def random_model():
    '''
    Returns a random sequence model where all logits are equal.
    '''
    input_chains = tf.keras.layers.Input(shape=(None,), name='input_ids', dtype='int32')
    input_ = tf.keras.layers.Input(shape=(None,), name='attention_mask', dtype='int32')
    batch, seq_len = tf.shape(input_chains)
    logits = tf.ones((batch, seq_len, 20), dtype=tf.float32)

    random_model = tf.keras.models.Model(inputs={'input_ids': input_chains,'attention_mask': input_},
                                         outputs={'logits': logits},
                                         name='Random Model')

    return random_model

A random sequence model is constructed below.

Recall, transformers tokenisers return two outputs (default) consisting of the tokenised sequence and an attention mask. To make the model compatible with DirectedEvolution we make the model accept these as well. *Note the attention mask does not change the returned probabilities.*

In [73]:
random_p_model = random_model()
print(random_p_model.summary())

Model: "Random Model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, None)]               0         []                            
                                                                                                  
 tf.compat.v1.shape_1 (TFOp  (2,)                         0         ['input_ids[0][0]']           
 Lambda)                                                                                          
                                                                                                  
 tf.__operators__.getitem_2  ()                           0         ['tf.compat.v1.shape_1[0][0]']
  (SlicingOpLambda)                                                                               
                                                                                       

<a id="section4"></a>

## Generate 100 variants of the short peptide using the random and blosum models

Below is code to generate 100 variants of the short peptide based on blosum model derived probabilities. Since the vocabulary for either case does not use a mask token we do not use masked marginals *as there is no masking*.

In [74]:
# Instantiate a weighted sampler.
variant_sampler = WeightedDirectedEvolutionSampler(temperature=1)
n_variants = 100
# Use the Directed Evolution sampler.
# Lets make 100 variants.
short_peptides = {
    'input_ids': tf.repeat(short_peptide, axis=0, repeats=n_variants),
    'attention_mask': tf.repeat(attention_mask, axis=0, repeats=n_variants)
}

variants_blosum, mutational_probabilities, logits = variant_sampler(sequence=short_peptides,
                                                                    model=blosum_p_model,
                                                                    max_steps=2, 
                                                                    mask_token=21, 
                                                                    batch=100, 
                                                                    masked_marginals=False)

variants_random, mutational_probabilities, logits = variant_sampler(sequence=short_peptides,
                                                                    model=random_p_model,
                                                                    max_steps=2,
                                                                    mask_token=21,
                                                                    batch=100, 
                                                                    masked_marginals=False)

2/2 [==============================] - 0s 160ms/Directed Evolution Steps


We can inspect the variant sequences generated.

In [75]:
print(tokeniser.batch_decode(variants_random[:10]))

['E G C', 'F A C', 'A M L', 'H V C', 'G C C', 'T A G', 'I M C', 'A A C', 'Q M C', 'E A G']


In [76]:
print(tokeniser.batch_decode(variants_blosum[:10]))

['T A A', 'R V C', 'S A C', 'A A R', 'A G G', 'H A F', 'S W C', 'W A A', 'A T R', 'N F C']
